Published on November 19, 2024. By Marília Prata, mpwolke

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Noam Chomsky on Language Aquisition

<iframe width="640" height="360" src="https://www.youtube.com/embed/7Cgpfw4z8cw" title="Noam Chomsky on Language Aquisition" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

"Language sets us apart other animals communicate but they don't have anything approaching the sophisticated grammar of human languages.  

"Young children become Adept in a new language very quickly since the dawn of philosophy thinkers have argued about whether or not we have innate idea is whether we 
are born knowing things as Plat believed. Or rather as John Lock and other empiricists argued the mind is a blank slate on which experience writes."

"The american linguist Chomsky gave a Twist to this debate in the 1960s by demonstrating that children learning to speak just don't have enough information to form the complex grammatical maneuvers that allow them to generate unlimited new and original sentences yet.

Chomsky's hypothesis was that there are inborn structures in our brain what he called a language acquisition device or LAD. Which gives us a natural propensity organize the spoken language that we hear in various grammatical ways. Without that, we couldn't get started as language Learners."go

https://www.youtube.com/watch?v=7Cgpfw4z8cw

In [ ]:
#Read One parquet file. Obviously, it's big.

train = pd.read_parquet("../input/wsdm-cup-multilingual-chatbot-arena/train.parquet")
train.tail(2)

## What languages we "speak" in this data.

In [ ]:
train['language']. value_counts()

In [ ]:
#Only Six rows of Hindi????

hindi = train[(train['language']=='Hindi')].reset_index(drop=True)
hindi.tail(10)

#train.loc[(train["language"] == "Hindi")]

## हेलो। नमस्कार। मैं बात करता हूँ sir

Hello, hello. I will talk to you Sir (Google translate)

In [ ]:
#On train row 41870 is 4th row on Hindi

train['prompt'][41870]

## Google Translate: Hello. Namaste. I am speaking to you sir. 

"Hello. Namaskar. I am speaking to you sir. Sir, I can hear your voice. Hello, yes, yes, sir Amar, I am speaking to you from Tarmiya sir. Sir, as we recently ran a campaign. Right of ORS and Banerjee. Yes, I have sent its report in the group sir. So, sir, we have shown that report on Facebook. Five thousand four hundred and thirty three times. Five thousand thirty four people have seen it from Ajmer. Total clicks on the post are thirty five and the click on your store link is twenty nine. Okay, so this is the detail sir related to your third campaign. So, further, your campaign is pending, so we will proceed on that as well. And then, as we talked to you, right, you had made payment for Lagera ji India Private Limited. Hmm hmm sir, so, like Mamta Medical Stores, we will merge it with this. Going further, like your campaigns which are pending, they will be provided to you in this."

![](https://yolainebodin.com/wp-content/uploads/2017/12/Quote-A-language-is-not-just-words...-Noam-Chomsky.png)Yolaine Bodin

#Keras installation

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook
# Install Keras 3 last. See https://keras.io/getting_started/ for more details.
!pip install -q -U keras-nlp
!pip install -q -U keras>=3
!pip install -q -U kagglehub --upgrade

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

os.environ["KERAS_BACKEND"] = "jax" # you can also use tensorflow or torch
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.00" # avoid memory fragmentation on JAX backend.
os.environ["JAX_PLATFORMS"] = ""
import keras
import keras_nlp
import kagglehub

## Kaggle Secrets

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

#Make yours and Add copy to clipboard
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("hf_licorne")

#Gabriel's line
#from kaggle_secrets import UserSecretsClient
#user_secrets = UserSecretsClient()
#os.environ["KAGGLE_USERNAME"] = user_secrets.get_secret("kaggle_username")
#os.environ["KAGGLE_KEY"] = user_secrets.get_secret("kaggle_key")

from tqdm.notebook import tqdm
tqdm.pandas() # progress bar for pandas

import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, Markdown

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

class Config:
    seed = 42
    dataset_path = "/kaggle/input/wsdm-cup-multilingual-chatbot-arena"
    preset = "gemma2_2b_en" # name of pretrained Gemma 2
    sequence_length = 512 # max size of input sequence for training
    batch_size = 1 # size of the input batch in training
    lora_rank = 4 # rank for LoRA, higher means more trainable parameters
    learning_rate=8e-5 # learning rate used in train
    epochs = 5 # Original is 10 number of epochs to train

In [ ]:
keras.utils.set_random_seed(Config.seed)

In [ ]:
#Don't change anything on Template line. Just the rows (in blue)
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

template = "\n\nCategory:\nkaggle-{Category}\n\nQuestion:\n{Question}\n\nAnswer:\n{Answer}"
hindi["prompt"] = hindi.apply(lambda row: template.format(Category=row.model_a,
                                                             Question=row.prompt,
                                                             Answer=row.response_a), axis=1)
data = hindi.prompt.tolist()


## Template utility function

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

def colorize_text(text):
    for word, color in zip(["Category", "Question", "Answer"], ["blue", "red", "green"]):
        text = text.replace(f"\n\n{word}:", f"\n\n**<font color='{color}'>{word}:</font>**")
    return text

#Gemma Causal

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

gemma_causal_lm = keras_nlp.models.GemmaCausalLM.from_preset(Config.preset)
gemma_causal_lm.summary()

#Define the specialized class

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

class GemmaQA:
    def __init__(self, max_length=512):
        self.max_length = max_length
        self.prompt = template
        self.gemma_causal_lm = gemma_causal_lm
        
    def query(self, category, question):
        response = self.gemma_causal_lm.generate(
            self.prompt.format(
                Category=category,
                Question=question,
                Answer=""), 
            max_length=self.max_length)
        display(Markdown(colorize_text(response)))

In [ ]:
x, y, sample_weight = gemma_causal_lm.preprocessor(data[0:2])

In [ ]:
print(x, y)

#Perform fine-tuning with LoRA

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

# Enable LoRA for the model and set the LoRA rank to the lora_rank as set in Config (4).
gemma_causal_lm.backbone.enable_lora(rank=Config.lora_rank)
gemma_causal_lm.summary()

#Gemma_causal_lm

Epochs!

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

#set sequence length cf. config (512)
gemma_causal_lm.preprocessor.sequence_length = Config.sequence_length 

# Compile the model with loss, optimizer, and metric
gemma_causal_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=Config.learning_rate),
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

# Train model
gemma_causal_lm.fit(data, epochs=Config.epochs, batch_size=Config.batch_size)

In [ ]:
gemma_qa = GemmaQA()

## Verify some rows 

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

row = hindi.iloc[4]
gemma_qa.query(row.model_a,row.prompt)

In [ ]:
row = hindi.iloc[2]
gemma_qa.query(row.model_a,row.prompt)

In [ ]:
row = hindi.iloc[0]
gemma_qa.query(row.model_a,row.prompt)

In [ ]:
row = hindi.iloc[5]
gemma_qa.query(row.model_a,row.prompt)

## The Languages We Speak Affect Our Perceptions of the World

Author: O.YU. MYKHAILYUK, H.YA. POHLOD

"The article presents review of studies on interaction between language and thought and the way it influences our understanding of the world. This is an interesting and important new direction for the research on language and thought, as our society becomes increasingly global and has an increased need for clear communication across languages and cultures. With a better understanding of how language and thought interact to influence how we understand and communicate about the world, we may begin to better understand how to communicate across languages and cultures, in ways beyond simple translation, that allow us to understand the various nuances of different languages and cultures." 

"Everyone who is multilingual talks about languages changing the way they think, affecting their personality. Each language is another personality as each language has its own distinctive way of expressing ideas. Varying grammar structures force the speaker to rethink how they emphasise certain ideas, and words can have different etymologies which, even if only on a subconscious level, affects the associations you have with them."

"a) Are you  unable to  think about  things you  don't have words for, or  do you lack words for them because you don't think about them? Part of the problem is that there is more involved than just language and thought; there is also culture. Your culture—the traditions, lifestyle, habits, and so on that you pick up from the people you live and interact with—shapes the way you think, and also shapes the way you talk."

"b) Are cultural practices a product of language, or is language a product of cultural practices? No matter what the case may be, learning another language is the best way to break free from whatever psychological shackles your mother tongue has placed on you.  What  is  clear  from  the  vast  research  literature  on  language  and  thought  is  that  language  and thought seem to  interact in some  way to  influence  how  we  understand  and communicate about  the world. 

"Moreover, if the new **language is very different from your own**, it may give you **insight into another culture** and **another way of life**." 

https://www.researchgate.net/publication/283783768_The_Languages_We_Speak_Affect_Our_Perceptions_of_the_World

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

category = "gemma-2-2b-it" #model_a
question = "हेलो। नमस्कार। मैं बात करता हूँ sir"  #Hello. Hello. I will talk to you sir
gemma_qa.query(category,question)

### Prompt hindi index2

उस समुच्चय का चयन करें जिसमेंसंख्याएँ उसी प्रकार संबंधित हैं जिस प्रकार निम्नलिखित समुच्चयोंकी\nसंख्याएँ संबंधित हैं।\n(55, 11, 25)\n(64,16, 16)

Google Translate

Select the set in which the numbers are related in the same way as the numbers of the following sets.\n(55, 11, 25)\n(64,16, 16)

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

category = "chatgpt-4o-latest-20240903" #model_a row index2
question = "उस समुच्चय का चयन करें जिसमेंसंख्याएँ उसी प्रकार संबंधित हैं जिस प्रकार निम्नलिखित समुच्चयोंकी\nसंख्याएँ संबंधित हैं।\n(55, 11, 25)\n(64,16, 16)"  #How many cover arts do we have?
gemma_qa.query(category,question)

### Prompt hindi index1

आपका नया नया जब लगे अभी आपकी सैलरी कम है लेकिन उसमें से भी आप हर महीने दो-तीन ₹4000 बच्चा रहे हो क्योंकि आगे जाके या तो आपको कर लेनी है या घरवालों के लिए घर लेना है उसका डॉ 

Google Translate:

When you feel that your salary is low right now, but even from that you are saving ₹4000 every month because later you will either have to do business or buy a house for your family.

In [ ]:
#By Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook

category = "jamba-1.5-mini" #model_a row index 1
question = "आपका नया नया जब लगे अभी आपकी सैलरी कम है लेकिन उसमें से भी आप हर महीने दो-तीन ₹4000 बच्चा रहे हो क्योंकि आगे जाके या तो आपको कर लेनी है या घरवालों के लिए घर लेना है उसका डॉ "  #How many cover arts do we have?
gemma_qa.query(category,question)

#Save the model

In [ ]:
preset_dir = ".\gemma2_2b_en_kaggle_docs"
gemma_causal_lm.save_to_preset(preset_dir)

#Acknowledgements:

Gabriel Preda https://www.kaggle.com/code/gpreda/fine-tuning-gemma-2-model-using-lora-and-keras/notebook